# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import csv
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
citiesDF = pd.read_csv("../Weatherpy/output_data/cities.csv")
citiesDF.dropna()
citiesDF.head()

,Unnamed: 0,City,Country,Date,Max Temp,Cloudiness,Wind Speed,Humidity,Latitude,Longitude
0,0,Saskylakh,RU,1619474685,2.98,97,4.52,97,71.9167,114.0833
1,1,Iquique,CL,1619474447,68.00,0,12.66,68,-20.2208,-70.1431
2,2,Qaanaaq,GL,1619474685,39.36,92,7.27,70,77.4840,-69.3632
3,3,Atuona,PF,1619474685,79.72,73,15.32,76,-9.8000,-139.0333
4,4,Butaritari,KI,1619474686,83.32,90,12.93,75,3.0707,172.7902


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key = g_key)

In [18]:
# Store lat and long into locations
humidityStored = citiesDF["Humidity"].astype(float)
locations = citiesDF[["Latitude", "Longitude"]]

In [28]:
# Plot heatmap
#figure_layout = {
#    'width': '800px',
#    'height': '600px'
#}

#fig = gmaps.figure(layout = figure_layout)
fig = gmaps.figure()

# Create and add heat layer
#heat_layer = gmaps.heatmap_layer(locations, dissipating = False, max_intensity = 100, point_radius = 3)
heat_layer = gmaps.heatmap_layer(locations, weights = humidityStored, dissipating = False, max_intensity = 100, point_radius = 1.5)

fig.add_layer(heat_layer)

# Display
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
narrowedDF = citiesDF[(citiesDF['Max Temp'] >= 65) & (citiesDF['Max Temp'] <=80)]
narrowedDF = narrowedDF[narrowedDF['Wind Speed'] < 10]
narrowedDF = narrowedDF[narrowedDF['Cloudiness'] == 0]

narrowedDF.head(10)

,Unnamed: 0,City,Country,Date,Max Temp,Cloudiness,Wind Speed,Humidity,Latitude,Longitude
34,34,Nāhan,IN,1619474695,73.69,0,5.91,21,30.5500,77.3000
95,95,Pisco,PE,1619474714,68.00,0,9.22,83,-13.7000,-76.2167
212,212,Chirongui,YT,1619474750,78.80,0,2.30,89,-12.9319,45.1483
254,254,Karratha,AU,1619474764,74.28,0,8.77,36,-20.7377,116.8463
259,259,Jardim,BR,1619474766,68.99,0,5.70,51,-21.4803,-56.1381
379,379,Diego de Almagro,CL,1619474802,67.28,0,5.57,43,-26.3667,-70.0500
441,441,Kalāleh,IR,1619474821,66.20,0,4.61,82,37.3807,55.4916
443,443,Garça,BR,1619474822,78.80,0,3.44,57,-22.2106,-49.6561
450,450,Vallenar,CL,1619474824,72.23,0,5.53,44,-28.5708,-70.7581
469,469,Machalí,CL,1619474830,69.80,0,5.75,37,-34.1833,-70.6667


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
baseURL = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [22]:

hotel_df = narrowedDF.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# params dictionary to update each iteration
params = {
    "radius": 50000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # get lat, lng 
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Try - Except because I kept getting a key error
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Data not found. Skipping.")
        
print ('------------------------')
print ('Data Retrieval Complete')
print ('------------------------')

{
    "html_attributions": [],
    "next_page_token": "ATtYBwLckt-0Y4Pn88jTcycndJGDROfdMsF5G0Tq9Ape0EnxTyNcCxRaTpXBciI6jOjZC-rcuG4Y_gr3YicFbyEN9kc6u-W9zP0RMEQSJnx7U1iOlk_ZOt58vIQhH83VKM6nUg4YuqWizLYQgOutK27AUzuhFBNe11JwKGEk_BEkXn1sh49HYifdDL-sZqWVM_XJSdSTp8R8tdxjlT42yeanFuvk3j5gwNDnY7f6u-UnDChqrCunPxGhALsQ58nbUgM4OeWDT-A41HZylKdCIblglms84oEI9eMhBauR4flbfM-Ly8jflVr16kk3qQFqbqMR4kOpzAHY3Xm6OotV5cT0ddG66QmetRGJs8Ha9MujaUc4YqgVPJjlBPI5mD6fOrLUmHgr8db4_MFVMw9qWXsLqRmiq3SVKDA-PuXHRm4fr5npI53TL0dgbt5CRRIq2rfw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 30.8186961,
                    "lng": 76.9907358
                },
                "viewport": {
                    "northeast": {
                        "lat": 30.82029837989272,
                        "lng": 76.99264887989273
                    },
                    "southwest": {
                        "lat": 30.817598720

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKlZhl1m52Ue-d_ABbapMgfvrRrMUlnBQ7Fs12I07v_hnDSSvJzh5KsD6OF27pN4dDtluPcnqcSUFnXAHguhSYRNVeXPuBHHJ82S5GtfFYXCoXxBOslAO-b9BXsC1zbn3fNaWg0KoJ12eQn5oGyg-28klqqbAp1jvDSUxrnR5eaKm5NBK1TkwpkML9mByfRTkaYLYGwkGnchZN8FtzZTouQ4R8rhoZo6dzi2u5oj6HWOAI1DfLQXEFOKNipPEgQEkd2qMQhfA9OqatD2IoUkBuxlXFYDyYkQmG7hXIKJ7vELkXsRbfwO9A0YZU38QrvP6drW9UVLxP2Q7QXjrjV4Rd5fxyLf9T1m_xRp8w5MMteTNxbfeMcnCuUgzyms8q9VbrmrOdYMVCydqcyYpcgoXojJWH8SbCK_-R_8kGZczDABEYnzgRcSfOtpJOD6P8y",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -13.8362619,
                    "lng": -76.2541983
                },
                "viewport": {
                    "northeast": {
                        "lat": -13.83495102010728,
                        "lng": -76.25282897010729
                    },
                    "southwest": {
                        "lat": -13.8376

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJ6DT0Jmz6XSkdRC1kKLQBE_7cKnOTqj6lP52WB5WRiBtiaYnyVzw73tn0N3rxLns1dJTEpBDlGfPiLfHU-wTwhMAFuW68gHQNtKDrhb9TSR9p5ybju-HChduVDV5m2MVg_msm0JD_SBCQQz8JZTuuc0k5pbaU_LzNMkv-eWiM20zqnVCwhJEgGe235T7rLNfwwmhfgg7HhLZTIZ307-R0yFuU699mSPdscnirrPjkCGAO44ouX-lWj5kbTf8Ra99hUW4O4DZ9xdVJp0p5sPtEuTRyMkgrRNszOZzNuOkPxFga6fpBYrs_OfaHL2RWcl8r3FBetTb7N2PLms8pXL5DbJd5Kv4DH8MZLU-SKPzypxpNXqsnAHZp8GcPbgqnRogv1R3kaNdpKc40cfaL_sTmj6UNGQCeH5XzimXeioObh_R0ld8pzJGvnHyLYuYuE",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -12.9617227,
                    "lng": 45.08435619999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -12.96014812010728,
                        "lng": 45.08580862989271
                    },
                    "southwest": {
                        "lat": -12

{
    "html_attributions": [],
    "next_page_token": "ATtYBwL2XeMK9-ILfOIkwBDjXD1c5bpxsk6Vylf5lq6Kde1QZNmsw0tCke0PhEUrOrumgmrlVaQcvClq-9FstT69MH2KypdujGlzaH94t0Gr6rA7IoY2_vwsYAQtVuG-NAXmndaLMr7KF52Ebw04UStL7HVJqUcCOnDYEzL_bLssWNkk-zP6VyFzm3KJr_xquvQ98fe04umwjv1lfEpyb2DAFGpbilDW_2LqnmucIVmsSO4KYtsDA0wRwVSa4fHi28nijKNV2xTojSeWOa83Lm6pi3DRo_zx7aCOoQE-N1q9HeWDDuyao_8Yv0M3wFtfUWeYyBcy0xfupDCicvZLmvEKx5S1beSk_fvpamnG_CEIn5Up5N0Uvy_ShunvI20hENFqq8HnOsLFLpjKOyjaLaaoB9eylwPAHWk6IpFkA5YVUHRnCuAuhXqDowoNlUke9iHr",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.7376386,
                    "lng": 116.8419976
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.73629782010728,
                        "lng": 116.8431256298928
                    },
                    "southwest": {
                        "lat": -20.73899

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJRsJ2_WnN1FCx5fta9xP2-GpC7x9VufbbNZSeHizSNHbtb9lp1m6pOObvSPGUgPm4kBd65D798isQvSA7LG0dgjXVNiV2ODdxkxxVWMYkT2GxI-osHHnh5q1kXsaMBWJ0EBvN-5UvMM3Rqjnfg_Ilb8cJ7vhsgKGDZ2-h3DWQia_XfB_qW6AzGCNwCNoF8swSo43oAiBZL9YlqKnbXm2GjFwa_G34551I_6Oy84sAPlxcj-tV-XBuajbsdvrBJbUDfaSQ5pjSLXn4wmPjcPENxGkoTzALBqJwp8aMuyV6Y2iQLHNZl8a7uUSpmYqqZssjlZiyfB_QbhGi9qDWuuORtgZcMZ-1JBMkMRwj8Eq0y9u8bLWwWfkaTs9w8MdFqBkaHr-xZ24aitaOCJeEVNngc7N1W7knmiifErM6fHHMlq9DtxzsAuzNDGwHfJAE7",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -21.109918,
                    "lng": -56.517364
                },
                "viewport": {
                    "northeast": {
                        "lat": -21.10842367010728,
                        "lng": -56.51634372010728
                    },
                    "southwest": {
                        "lat": -21.111123

...

{
    "html_attributions": [],
    "next_page_token": "ATtYBwIncciOuQNDF1mCAcXJYndeM9Bsmoh4kSyE1Jm63r1Jeei8VZchpguZFNTxIFqqK4SHpQLEw3Tw9oMpoY0VItUU80T3j3KT4fl7RmcEcihzoJ2fpxb-sO59xqxpuddMzPPO-o1V9q-NNQKVI_d5nonDc0nroyfk45Oxv9FpAIKbCL79U2rphnFGGC7pTPMhV7d2iuVTCxNDWkLgqktBiOE4p66VYD3pWVOFxc3GAMWeEC4vFJW5bmq_3QD34pobgsU0zmgBYGv2WFZ7XmXw-u4X1t9N9v0ZRlw7sw9mklY4xqJ_e-HyOn6oS0bpc5ockAjmjWuUJ7uIYG6K7_e_bYw7gGbozUQbcrbOg6y-4pR_c2iZ272vu6Q-uRJPmvxFH0RAzszY7_rBBfFxNQF5JwQVKYC6kb7vNO3dYlQgbnacKkP8zEtfPlsbpYoQVpaN",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 37.0988823,
                    "lng": 55.1725084
                },
                "viewport": {
                    "northeast": {
                        "lat": 37.10023567989273,
                        "lng": 55.17397862989272
                    },
                    "southwest": {
                        "lat": 37.097536020

{
    "html_attributions": [],
    "next_page_token": "ATtYBwKcCiYdDnkynmJx0UOPlyK3kUrFhh93XYnvO10n_vc5ryycpQ_BUG_q9FIqKasC8oGSyyNhvJH9CyfSbkbBdETzw6ZAyVwHAw7yNL3h2NIeSb018stM2LGclT18rwKG8i46QVa7JIus6yMe_18yI2xtkc6cIQVRihQ7Qvn-2AHTBrg6T4Z63wqvg7amqg97nDAWCY2cT2ZZUgk4gJqyX0g4nTEYV3bMoXjvr1LOUiVq4eM8SYIuTCX4npvCfdbELBPLWrnSyad0g9_V4XtE01Q0OS6rHlF8YBbx94ZNZ_mZYwiGOvtsENChA5Z48H9JoqTiwG5p2mDsitsOLDYDVotVmCuKpWRdwgEGheWfTFIaDD30mtVCccIvyqX1pU2Fmwyqr849nMB5sQzfABZARZEjYzQc-k2dP_M2rsGgnsZ13cCuP-fkvoRjNarkRiE8",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -22.227228,
                    "lng": -49.9584734
                },
                "viewport": {
                    "northeast": {
                        "lat": -22.22596287010728,
                        "lng": -49.95730882010727
                    },
                    "southwest": {
                        "lat": -22.22866

{
    "html_attributions": [],
    "next_page_token": "ATtYBwITaqvOHm8dYy2swMLByjFhT_-iIKk2pAurlNENObRUS_bS6lJSPVQWeiixe2HqHy_y49V5E6bP_yKJvzIIUB8XBsdDEkbi-PNb6LhWtEEQ03C241hpsBiZReGlIzlid6Ip2fgAlCxbQ7KrImjpRatItb7oEhMYqMNGbGGQ5Fdr7IE5nkfB4TpE3AEBkWOmEQ3IFbAJERqNFj7wKQk-nsK35ok8tKPydeC1z_uM4DGSas3d8ZwQQrGRMkPkijdEEHCutyqxCya20mfkwSd87PML_GEzSBLPn--Wx1ofsGVgepePjXO0isj1BEMIxUAiGFIbRJMMcreqyqfTgUQNEB8IWlFNw6OOP6Q6aG0Yuun5UPUQqiwqEpjmAXS1E4QfPe4QxCw6k0To5G5sVXUJ9GW-qygkzwE790dGx2LaPZUSWz4rwq0mvervAf31eQ4s",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -28.5781955,
                    "lng": -70.77928159999999
                },
                "viewport": {
                    "northeast": {
                        "lat": -28.57683037010728,
                        "lng": -70.77798977010727
                    },
                    "southwest": {
                        "lat": -

{
    "html_attributions": [],
    "next_page_token": "ATtYBwJFgn_qgD3bT6G_TZCRi3I4WgzgQQUJhS8PUeUN9WebjhAPn9yd-7e1QSSFOJgj-uioYjLIwq4pjiwT_IjEACob_cBOASLteVfkp_ivkh1G3so8WmDk3ukZa343zXfZkcq53gG7cHq-I6KY1lAvxJXGLACc-BkVjMG5IaZgC9WDA7YU5Z1GjeU76YMqUa242MB5iYHHRmSHkNJjtGZ9JuF0VaS-O4SzDxxHnjnbVQy7ofVjiLGruM0NJ9Op3Nhw_MNuFB2VJSyOLJocmNwt_eX79Yo2e1WCw6gLhVFIE9ScDoELke0EwGscdOrcjtp8dGybh13xUYcYglJGH8xGpHo8imVLQuH_3Y6ECf_OkC-NLDuFj08BUffgW7BJwEQpb0ZMgsY4jNpJBawZHBLz-ZbKi9fYqy0soo2DAoiRfmfOAnw0nNozK95_ARDRczkv",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -33.7548664,
                    "lng": -70.2797265
                },
                "viewport": {
                    "northeast": {
                        "lat": -33.75350182010728,
                        "lng": -70.27825172010728
                    },
                    "southwest": {
                        "lat": -33.7562

{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Data not found. Skipping.
{
    "html_attributions": [],
    "next_page_token": "ATtYBwJ0W5TqSQtLcwwP7v0T_xySGpC-3wkU7qdNDwh27T9oiFtU_9yvLuMJVg5rM_UZRRRh38uZRDBp9bg1CoMI1c27QqpxMfKdGudU1Ri_XVoBU9CKQnt-wDcHo992yonBm6VcQWZbtGHE61n93V6kcIwh8Kwgck7EW-XmL79Ekphqz60yBrjfKyBtledYBYM4UgyhFKgmB1lKEJ3mrLh6aenZmGNfUQsxzdzhL97e2KNnA9hx94EU4IeYM6TXEBS7YJqElUpSsE1PmSq13uREDZL17GxqHeaMWTJuC_b3pIszPMkJ-LylUrXjYg61_NRjVOsGKue7jen3WaoVG_fTYZWVIniNR6iuf-e5alyaBqx1osL30VZ8iPmrm4rGgxKUzf6Mv4eiodVWyKSVPSHgZzw14iwHEp8ArLbljfnDAcWTdpIRiW7zrgdCdsa18-Id",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": 12.4354675,
                    "lng": -16.7817321
                },
                "viewport": {
                    "northeast": {
                        "lat": 12.43681732989272,
                        "lng": -16.7803822

In [31]:
hotel_df

,Unnamed: 0,City,Country,Date,Max Temp,Cloudiness,Wind Speed,Humidity,Latitude,Longitude,Hotel Name
0,34,Nāhan,IN,1619474695,73.69,0,5.91,21,30.5500,77.3000,Moksha Himalaya Spa Resort
1,95,Pisco,PE,1619474714,68.00,0,9.22,83,-13.7000,-76.2167,"Hotel Paracas, a Luxury Collection Resort, Par..."
2,212,Chirongui,YT,1619474750,78.80,0,2.30,89,-12.9319,45.1483,Le Jardin Maoré
3,254,Karratha,AU,1619474764,74.28,0,8.77,36,-20.7377,116.8463,Karratha International Hotel
4,259,Jardim,BR,1619474766,68.99,0,5.70,51,-21.4803,-56.1381,Zagaia Eco Resort
5,379,Diego de Almagro,CL,1619474802,67.28,0,5.57,43,-26.3667,-70.0500,Camino Del Inca Hotel
6,441,Kalāleh,IR,1619474821,66.20,0,4.61,82,37.3807,55.4916,Hotel Akam
7,443,Garça,BR,1619474822,78.80,0,3.44,57,-22.2106,-49.6561,Quality Hotel & Conv. Center Marília
8,450,Vallenar,CL,1619474824,72.23,0,5.53,44,-28.5708,-70.7581,Hotel Puerto de Vega
9,469,Machalí,CL,1619474830,69.80,0,5.75,37,-34.1833,-70.6667,El Cielo Lodge


In [32]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = narrowedDF[["Latitude", "Longitude"]]

In [33]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))